In [1]:
import os
WORK = os.environ["WORK"]
PROJ_DIR = f'{WORK}/ADA_Project'
os.chdir(PROJ_DIR)

In [2]:
from glob import glob
import numpy as np
import PIL.Image
from tqdm import tqdm
from pathlib import Path
import urllib.request
import tarfile

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import tensorflow as tf
tf.compat.v1.enable_eager_execution()

import DeepFAMS

Tensorflow version: 1.14.0


In [3]:
RAW_IMGS_DIR, RESIZED_IMGS_DIR, DATA_CUSTOM_DIR, TRAIN_RUNS_DIR = DeepFAMS.utils.return_dirs(PROJ_DIR, 'StanfordDogs')

In [4]:
# url = 'http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar'

# DeepFAMS.utils.Get_Raw_Data(url=url,
#              datasets_dir=f'{PROJ_DIR}/datasets',
#              RAW_IMGS_DIR=RAW_IMGS_DIR,
#              file_name='StanfordDogs_images.tar')

Downloaded and extracted to:
/work/chaselab/malyetama/ADA_Project/datasets/StanfordDogs_images_raw/Images


In [8]:
raw_imgs = glob(f'{RAW_IMGS_DIR}/**/**/**/*')
print(len(raw_imgs))

20580


In [7]:
# for x in tqdm(raw_imgs):
#     DeepFAMS.preprocessing.resize_imgs(x, (256, 256), RESIZED_IMGS_DIR)

 73%|███████▎  | 15111/20580 [03:44<01:19, 68.73it/s]

cannot write mode RGBA as JPEG
Warning! Failed to save /work/chaselab/malyetama/ADA_Project/datasets/StanfordDogs_images_raw/Images/Images/n02105855-Shetland_sheepdog/n02105855_2933.jpg!
Deleting corrupted image...


100%|██████████| 20580/20580 [05:04<00:00, 67.50it/s]


In [9]:
print(f'Raw: {len(raw_imgs)}, Resized: {len(glob(f"{RESIZED_IMGS_DIR}/*"))}')

Raw: 20580, Resized: 20579


In [10]:
DeepFAMS.preprocessing.tf_record_exporter(tfrecord_dir=DATA_CUSTOM_DIR, image_dir=RESIZED_IMGS_DIR, shuffle=1)

Loading images from "/work/chaselab/malyetama/ADA_Project/datasets/StanfordDogs_resized"
Creating dataset "/work/chaselab/malyetama/ADA_Project/datasets/StanfordDogs_custom"
Added 20579 images.                     


In [20]:
with open('compile_model.sh', 'w') as f:
        f.write(f'''#!/bin/bash
module load anaconda
module load compiler/gcc/4.7
module load cuda
$WORK/.conda/envs/ada-env/bin/python $WORK/ADA_Project/StyleGAN2-ada__source_code/train.py \
--outdir={TRAIN_RUNS_DIR} \
--gpus=2 \
--data={DATA_CUSTOM_DIR} \
--snap=1 \
--kimg=1''')

In [21]:
DeepFAMS.utils.execute('cat compile_model.sh')

#!/bin/bash
module load anaconda
module load compiler/gcc/4.7
module load cuda
$WORK/.conda/envs/ada-env/bin/python $WORK/ADA_Project/StyleGAN2-ada__source_code/train.py --outdir=/work/chaselab/malyetama/ADA_Project/training_runs/StanfordDogs_training-runs --gpus=2 --data=/work/chaselab/malyetama/ADA_Project/datasets/StanfordDogs_custom --snap=1 --kimg=1


In [ ]:
DeepFAMS.utils.execute('bash compile_model.sh')

In [ ]:
os.remove('compile_model.sh')

In [ ]:
for num in range(-1, -10, -1):
    files = DeepFAMS.utils.last_snap(num, TRAIN_RUNS_DIR)
    if files != []:
        break

latest_snap = sorted(files)[-1]
print(latest_snap)

In [ ]:
run_desc, training_options = DeepFAMS.setup_training_options(
    gpus       = 2,
    snap       = 1,
    data       = DATA_CUSTOM_DIR,
    resume     = latest_snap
)

In [ ]:
DeepFAMS.utils.execute('nvidia-smi')

In [ ]:
DeepFAMS.RunTraining(outdir=TRAIN_RUNS_DIR, seed=1000,
             dry_run=True, run_desc=run_desc, training_options=training_options)

In [ ]:
tf.compat.v1.disable_eager_execution()
DeepFAMS.RunTraining(outdir=TRAIN_RUNS_DIR, seed=1000,
             dry_run=False, run_desc=run_desc, training_options=training_options)